# Apresentação ✒️

Notebook destinado ao estudo do framework LangChain. Nesse primeiro momento, irei estudar sobre modelos de uso, prompts e parses, que podem modulam a saída de resposta da LLM a partir do framework.

## Bibliotecas 📚

In [1]:
!pip install google-generativeai -q

In [2]:
!pip install langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 14.0 MB/s eta 0:00:00


In [3]:
!pip install langchain_google_genai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 10.4 MB/s eta 0:00:00


In [52]:
import warnings
warnings.filterwarnings('ignore')

import google.generativeai as genai

import os
import langchain

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import ChatPromptTemplate

from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [5]:
os.environ['GOOGLE_API_KEY'] = 'sua-api-key'

In [8]:
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

## Instanciando a LLM de uso 🤖

In [9]:
# Criando o modelo de llm utilizado com o google gemini.

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-pro-latest",
    temperature = 0.8
)

### Templates

Uma das vantagens que o framework oferece é elaborar templates (formatação) de mensagens, por meio das quais ajuda a prover a automatização de mensagens, modulando-as num formato desejado, segundo o seu contexto de uso.

Tomamos como exemplo um cliente que escreve um e-mail em resposta ao seu fornecedor pouco profissionai. O objetivo é transformar a sua mensagem em algo profissional, em ordem de não produzir antipatia, mas ainda informar a sua perspectiva acerca de sua insatisfação.

In [38]:
template_string = """Deixe o texto \
que está delimitado por três crases \
em um estilo que seja {style}. \
texto : ```{text}```
"""

In [39]:
customer_style = """ Português brasileiro num tom que seja \
calmo, respeitoso e profissional.
"""

In [40]:
customer_email = """Arrr, estou furioso porque a tampa do meu liquidificador \
saiu voando e espalhou smoothie nas paredes da minha cozinha! \
E para piorar as coisas, a garantia não cobre o custo de \
limpar minha cozinha. Preciso da sua ajuda \
agora mesmo, camarada!
"""

In [41]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [42]:
customer_messages = prompt_template.format_messages(
    style = customer_style,
    text = customer_email
)

In [43]:
print(customer_messages)

[HumanMessage(content='Deixe o texto que está delimitado por três crases \\ \nem um estilo que seja  Português brasileiro num tom que seja calmo, respeitoso e profissional.\n. texto : ```Arrr, estou furioso porque a tampa do meu liquidificador saiu voando e espalhou smoothie nas paredes da minha cozinha! E para piorar as coisas, a garantia não cobre o custo de limpar minha cozinha. Preciso da sua ajuda agora mesmo, camarada!\n```\n')]


In [35]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [36]:
# Requisitando a LLM para traduzir o texto do cliente
# para um formato que seja repeitoso.

customer_response = llm(customer_messages)

In [37]:
customer_response.content

'"Olá, boa tarde. Gostaria de manifestar minha frustração com um problema que ocorreu com meu liquidificador. A tampa se soltou durante o uso, causando um pequeno acidente na minha cozinha. Infelizmente, a garantia não cobre os custos de limpeza. Solicito gentilmente uma solução para este inconveniente. Agradeço a atenção." \n'

## Formatando saídas

Podemos formatar a saída que a LLM nos retorna, especificando um formato através do qual ela deve seguir. Por exemplo, essa abordagem é útil quando pretende-se regularizar as saídas difusas da LLM para um padrão mais regular, como formatação JSON, na qual por meio de chaves e seus valores, consegue-se compreender as informações geradas sob um ponto de vista sintético, mas igualmente informativo.

Nessa etapa irei querer compreender o review de um cliente sob 4 parâmetros determinados, estruturados numa formatação JSON, como a seguir :      

```
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
  "sentiment": "happy"
}
```

In [45]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
Para o seguinte texto extraia as seguintes informações :

presente: O item foi comprado como um presente para outra pessoa? \
Responda Verdadeiro se sim, Falso se não ou se desconhecido.

dias_entrega: Quantos dias levou para o produto \
chegar? Se essa informação não for encontrada, insira -1.

valor_preco: Extraia quaisquer frases sobre o valor ou preço, \
e coloque-as como uma lista Python separada por vírgulas.

sentimento_presente : Extraia o sentimento presente do texto, como \
positivo, negativo ou neutro, com uma breve explicação sobre \
cada qual, que ilustra o sentimento presente do cliente.

Formate a saída como JSON com as seguintes chaves e, se necessário, traduza \
o texto para português :

presente
dias_entrega
valor_preco
sentimento_presente

text: {text}
"""

In [46]:
prompt_review_template = ChatPromptTemplate.from_template(review_template)

In [48]:
print(prompt_review_template)
print("")
print(type(prompt_review_template))

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='Para o seguinte texto extraia as seguintes informações :\n\npresente: O item foi comprado como um presente para outra pessoa? Responda Verdadeiro se sim, Falso se não ou se desconhecido.\n\ndias_entrega: Quantos dias levou para o produto chegar? Se essa informação não for encontrada, insira -1.\n\nvalor_preco: Extraia quaisquer frases sobre o valor ou preço, e coloque-as como uma lista Python separada por vírgulas.\n\nsentimento_presente : Extraia o sentimento presente do texto, como positivo, negativo ou neutro, com uma breve explicação sobre cada qual, que ilustra o sentimento presente do cliente.\n\nFormate a saída como JSON com as seguintes chaves e, se necessário, traduza \\ \no texto para português :\n\npresente\ndias_entrega\nvalor_preco\nsentimento_presente\n\ntext: {text}\n'))]

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>


In [49]:
messages = prompt_review_template.format_messages(
    text = customer_review
)

In [50]:
customer_response = llm(messages)

In [51]:
customer_response.content


'```json\n{\n"presente": true,\n"dias_entrega": 2,\n"valor_preco": [\n"ligeiramente mais caro que os outros sopradores de folhas",\n"vale a pena pelo recursos extras"\n],\n"sentimento_presente": {\n"classificacao": "positivo",\n"explicacao": "O cliente expressa satisfação com o soprador de folhas, descrevendo-o como \'incrível\' e destacando suas funcionalidades. Ele também menciona que o produto chegou a tempo para o aniversário de sua esposa e que acredita que ela tenha gostado muito."\n}\n}\n```'

### Formatando a saída para um dicionário em Python

In [65]:
# Criando uma formatação de resposta para a saída da LLM.

gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")

price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

price_sentiment_schema = ResponseSchema(name = "sentiment",
                                        description="Extract the prevalent sentiment \
                                        from customer review, as positive, neutral or negative \
                                        with a short explication about it each one")


In [66]:
# Criando uma lista com os formatos que a LLM deverá seguir, em ordem
# de prover cada resposta na formatação correta.

response_schemas = [gift_schema,
                    delivery_days_schema,
                    price_value_schema,
                    price_sentiment_schema]

In [67]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [68]:
# Obtendo o formato de de intrução que a LLM deverá seguir.

format_instructions = output_parser.get_format_instructions()

In [74]:
review_template_2 = """\
Para o seguinte texto extraia as seguintes informações :

presente: O item foi comprado como um presente para outra pessoa? \
Responda Verdadeiro se sim, Falso se não ou se desconhecido.

dias_entrega: Quantos dias levou para o produto \
chegar? Se essa informação não for encontrada, insira -1.

valor_preco: Extraia quaisquer frases sobre o valor ou preço, \
e coloque-as como uma lista Python separada por vírgulas.

sentimento_presente : Extraia o sentimento presente do texto, como \
positivo, negativo ou neutro, com uma breve explicação sobre \
cada qual, que ilustra o sentimento presente do cliente.

texto : {text}

{format_instructions}

Faça com que o idioma do texto de saída seja português.
"""

In [75]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [76]:
response = llm(messages)

In [77]:
response.content

'```json\n{\n\t"gift": "True",\n\t"delivery_days": "2",\n\t"price_value": "[\'É um pouco mais caro do que os outros sopradores de folhas por aí, mas acho que vale a pena pelos recursos extras.\']",\n\t"sentiment": "positivo - O cliente expressa satisfação com o produto, descrevendo-o como \'incrível\' e mencionando que sua esposa gostou muito. Ele também destaca os aspectos positivos do produto, como as diferentes configurações e a eficácia na limpeza das folhas."\n}\n```'

In [78]:
output_dict = output_parser.parse(response.content)

In [79]:
# Gerando a resposta numa saída em dicionário Python

output_dict

{'gift': 'True',
 'delivery_days': '2',
 'price_value': "['É um pouco mais caro do que os outros sopradores de folhas por aí, mas acho que vale a pena pelos recursos extras.']",
 'sentiment': "positivo - O cliente expressa satisfação com o produto, descrevendo-o como 'incrível' e mencionando que sua esposa gostou muito. Ele também destaca os aspectos positivos do produto, como as diferentes configurações e a eficácia na limpeza das folhas."}